In [ ]:
#
# This file contains the common functions and definitions used in the Egeria Hands on
# Lab Notebooks.
#

# These functions define the location of Coco Pharmaceutical's OMAG Server Platforms
# where the metadata servers and governance servers run.  They test for the presence
# of the environment variables that define the platform network addresses in the
# Docker and Kubernetes runtime environments.  If the environment variables are not
# present then the localhost defaults are used.

import os

corePlatformURL     = os.environ.get('corePlatformURL','http://localhost:8080') 
dataLakePlatformURL = os.environ.get('dataLakePlatformURL','http://localhost:8081') 
devPlatformURL      = os.environ.get('devPlatformURL','http://localhost:8082')

# The OMAG Server Platforms host one to many OMAG Servers.  An OMAG Server could be
# a metadata server or a specialized governance server.  Its behavior is determined
# by a configuration document that defines which OMAG services are activated.
# All OMAG Server Platforms support the administration commands to define a server's
# configuration document.  It is also possible to create configuration documents
# through admin calls to one OMAG Server Platform and then deploy them to the
# OMAG Server Platform where they are to run.  In the Egeria hands on lab, the
# OMAG Server configuration is created on the dev platform and deployed to the
# production platforms as needed.

adminPlatformURL = devPlatformURL

# Gary Geeke is the IT Administration Lead at Coco Pharmaceuticals.
# He does all of the configuration for the OMAG Servers.

adminUserId = "garygeeke"


# The OMAG Server Platform is a single executable (application) that can be started
# from the command line or a script or as part of a pre-built container environment
# such as docker-compose or kubernetes.  The function below checks that a specific
# server platform is running.

import requests
import pprint
import json

def checkServerPlatform(testPlatformName, testPlatformURL):
    isPlatformActiveURL = testPlatformURL + "/open-metadata/platform-services/users/" + adminUserId + "/server-platform/origin"
    response = requests.get(isPlatformActiveURL)
    if response.status_code == 200:
        print("   ", testPlatformName, "is active")
        return True
    else:
        print("   ", testPlatformName, "is down - start it before proceeding")
        return False


# The OMAG Server Platform has the implementation of the open metadata and governance (OMAG) services.  These are
# divided into three groups: Common Services, Access Services and Governance Services.

def printServiceDescriptions(testPlatformName, serviceGroupName, services):
    print(serviceGroupName, "for", testPlatformName)
    for x in range(len(services)):
        serviceName = services[x].get('serviceName')
        serviceDescription = services[x].get('serviceDescription')
        serviceWiki = services[x].get('serviceWiki')    
        print (" * ", serviceName)
        print ("     ", serviceDescription)
        print ("     ", serviceWiki)
    print (" ")
    
def getServerPlatformServices(testPlatformName, testPlatformURL):
    getOMAGServicesURL = testPlatformURL + "/open-metadata/platform-services/users/" + adminUserId + "/server-platform/registered-services"
    response = requests.get(getOMAGServicesURL + "/common-services")
    if response.status_code == 200:
        printServiceDescriptions(testPlatformName, "Common services", response.json().get('services'))
    response = requests.get(getOMAGServicesURL + "/access-services")
    if response.status_code == 200:
        printServiceDescriptions(testPlatformName, "Access services", response.json().get('services'))
    response = requests.get(getOMAGServicesURL + "/governance-services")
    if response.status_code == 200:
        printServiceDescriptions(testPlatformName, "Governance services", response.json().get('services'))


# Each server is configured to define which services should be actived.  This configuration results in
# the creation of a configuration document.  This document is read when the server is started and
# drives the initialization of the services.

def checkServerConfigured(testServerName, testPlatformName, testPlatformURL):
    isServerKnownOnPlatform = testPlatformURL + "/open-metadata/admin-services/users/" + adminUserId + "/servers/" + testServerName + "/configuration"
    response = requests.get(isServerKnownOnPlatform)
    if response.status_code == 200:
        serverConfig=response.json().get('omagserverConfig')
        auditTrail = serverConfig.get('auditTrail')
        if auditTrail is not None:
            print("           ...", testServerName, "is configured")
            return True
        else:
            print("           ...", testServerName, "needs configuring")
    else:
        print("   ...", testPlatformName, "at", testPlatformURL, "is down - unable to check server configuration")
        return False


# The OMAG Server Platform supports a call to return if a server is active.

def checkServerActive(testServerName, testPlatformName, testPlatformURL):
    isServerActiveOnPlatform = testPlatformURL + "/open-metadata/platform-services/users/" + adminUserId + "/server-platform/servers/" + testServerName + "/status"
    response = requests.get(isServerActiveOnPlatform)
    if response.status_code == 200:
        serverStatus = response.json().get('active')
        if serverStatus == True:
            print("           ...", testServerName, "is active - ready to begin")
        else:
            print("           ...", testServerName, "is down - needs to be started")
        return serverStatus
    else:
        print("   ...", testPlatformName, "at", testPlatformURL, "is down - unable to check server configuration")
        return False

    
# This is the call to start a server on a specific platform.  Once the server is running it is possible to
# make use of the open metadata and governance services that have been activated in the server.

def activateServerOnPlatform(testServerName, testPlatformName, testPlatformURL):
    print ("                Starting server " + testServerName + " ...")
    activateServerURL = testPlatformURL + "/open-metadata/admin-services/users/" + adminUserId + '/servers/' + testServerName + "/instance"
    response = requests.post(activateServerURL)
    if response.status_code == 200:
        return True
    else:
        print ("                   ..." + testServerName + " failed to start")
        return False


# Once a server is active, it is possible to query the services that are active.

def getServerServices(testServerName, testPlatformName, testPlatformURL):
    getServerActiveServicesURL = testPlatformURL + "/open-metadata/platform-services/users/" + adminUserId + "/server-platform/servers/" + testServerName + "/services"
    print ("Services for server:", testServerName)
    response = requests.get(getServerActiveServicesURL)
    if response.status_code == 200:
        serviceList = response.json().get('serverServicesList')
        for x in range(len(serviceList)):
            print (" * ", serviceList[x])
    else:
        print (response)

# This function checks whether a server is active and starts it if it is down.

def activateServerIfDown(testServerName, testPlatformName, testPlatformURL):
    print("        Checking OMAG Server " + testServerName)
    configured = checkServerConfigured(testServerName, testPlatformName, testPlatformURL)
    if configured == True:
        active = checkServerActive(testServerName, testPlatformName, testPlatformURL)
        if active == False:
            activateServerOnPlatform(testServerName, testPlatformName, testPlatformURL)


def activatePlatform(testPlatformName, testPlatformURL, hostedServerNames):
    available = checkServerPlatform(testPlatformName, testPlatformURL)
    if available == True:
        for x in range(len(hostedServerNames)):
            activateServerIfDown(hostedServerNames[x], testPlatformName, testPlatformURL)
        
        

print("\nChecking OMAG Server Platform availability...")

activatePlatform("Core Platform", corePlatformURL, ["cocoMDS2", "cocoMDS3", "cocoMDS5", "cocoMDS6"])
activatePlatform("Data Lake Platform", dataLakePlatformURL, ["cocoMDS1", "cocoMDS4"])
activatePlatform("Dev Platform", devPlatformURL, ["cocoMDSx"])

print ("Done.")

# print (" ")
# getServerPlatformServices("Dev Platform", devPlatformURL)
# print (" ")
# getServerServices("cocoMDS1", "Data Lake Platform", dataLakePlatformURL)
# print (" ")
# getServerServices("cocoMDS4", "Data Lake Platform", dataLakePlatformURL)


# The open metadata servers are linked together through the following open metadata cohorts.
# The servers linked via a cohort can exchange open metadata either through federated
# queries or metadata replication.

cocoCohort = "cocoCohort"
devCohort  = "devCohort"
iotCohort  = "iotCohort"
ctsCohort  = "ctsCohort"